In [ ]:
import numpy as np
import os
import cv2
import torch
from torch.optim import Adam
from tqdm import tqdm
from model import YOLOV5m
from utils.validation_utils import YOLO_EVAL
from utils.training_utils import get_loaders
from ultralytics_loss import ComputeLoss

from utils.plot_utils import cells_to_bboxes, plot_image
from utils.bboxes_utils import non_max_suppression

import matplotlib.pyplot as plt

In [ ]:
nc = 4
labels = ['wood', 'glass', 'plastic', 'metal']
first_out = 48
ANCHORS = [
    [(10, 13), (16, 30), (33, 23)],  # P3/8
    [(30, 61), (62, 45), (59, 119)],  # P4/16
    [(116, 90), (156, 198), (373, 326)]  # P5/32#
]
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model = YOLOV5m(first_out=first_out, nc=nc, anchors=ANCHORS,
                ch=(first_out * 4, first_out * 8, first_out * 16), inference=False).to(device)

In [ ]:
model.load_state_dict(torch.load('yolov5_v5_640_640.pth'))

In [ ]:
# model = model.to('cpu')
# torch_input = torch.randn(1, 14, 640, 640)
# export_output = torch.onnx.export(model, torch_input, 'yolov5_v6_640_640.onnx')

In [ ]:
optim = Adam(model.parameters(), lr=0.00001)

In [ ]:
optim.load_state_dict(torch.load('yolov5_v5_640_640_optim.pth'))

In [ ]:
save_logs = False
rect_training = True
filename = "model_hack"
resume = False
loss_fn = ComputeLoss(model, save_logs=save_logs, filename=filename, resume=resume)

In [ ]:
conf_threshold = 0.6
nms_iou_thresh = 0.7
map_iou_thresh = 0.7
evaluate = YOLO_EVAL(save_logs=save_logs, conf_threshold=conf_threshold,
                         nms_iou_thresh=nms_iou_thresh,
                         map_iou_thresh=map_iou_thresh,
                         device=device, filename=filename, resume=resume)

In [ ]:
root_path = 'C:\\Users\\Safiu\\Projects\\PHO_hack\\data\\'
train_loader, val_loader = get_loaders(db_root_dir=root_path, batch_size=8, num_classes=nc,
                                           box_format='yolo', ultralytics_loss=True,
                                           rect_training=rect_training, num_workers=1)

In [ ]:
best_cls_acc = 0
best_obj_acc = 0
for epoch in range(1,500):
        print(epoch)
        model.train()

#         loop = tqdm(train_loader)
        avg_batches_loss = 0
        loss_epoch = 0
        nb = len(train_loader)
        optim.zero_grad()
        idx = 0
        for images, bboxes in train_loader:
            images = images.float() / 255


            images = images.to(device)
            bboxes = bboxes.to(device)
            # BBOXES AND CLASSES ARE PUSHED to.(DEVICE) INSIDE THE LOSS_FN

            # If I had a V100...
#             with torch.cuda.amp.autocast():
            out = model(images)
            loss = loss_fn(out, bboxes, pred_size=images.shape[2:4], batch_idx=idx, epoch=epoch)
            avg_batches_loss += loss
            loss_epoch += loss

            # backpropagation
            # check docs here https://pytorch.org/docs/stable/amp.html
            loss.backward()
            optim.step()
            idx = idx +1

        print(f"==> training_loss: {(loss_epoch.item() / nb):.2f}")

        model.eval()

        cls_acc, obj_acc = evaluate.check_class_accuracy(model, val_loader)
        evaluate.map_pr_rec(model, val_loader, anchors=model.head.anchors, epoch=epoch)

In [ ]:
# model.eval()
# torch.save(optim.state_dict(), 'yolov5_v6_640_640_optim.pth')
# torch.save(model.state_dict(), 'yolov5_v6_640_640.pth')

In [ ]:
model.eval()
# model = model.to(device)
test_arr = np.load('3000.npy')
test_arr = np.expand_dims(test_arr, 0)
test_arr = np.float32(test_arr)/255
test_arr = torch.from_numpy(test_arr)
test_arr = test_arr.to(device)
with torch.no_grad():
    out = model(test_arr)

In [ ]:
bboxes = cells_to_bboxes(out, model.head.anchors, model.head.stride, is_pred=True, to_list=False)
bboxes = non_max_suppression(bboxes, iou_threshold=0.6, threshold=0.2, tolist=False)

In [ ]:
len(bboxes)

In [ ]:
test_arr = np.load('3000.npy')
test_arr = test_arr[0:3].copy()
test_arr = np.moveaxis(test_arr, 0, -1)
test_arr = np.uint8(test_arr)
plt.imshow(test_arr)

In [ ]:
bboxes = bboxes.cpu().detach()
plot_image(test_arr, bboxes, labels)